# 전이 학습 예제(Cats ans Dogs)

## <span style="font-size:25px">Fine-Tuning using Xception model</span>

<img src="picture/TF_img_26_1.png" alt="TF_img_26_1" width=850>

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_WIDTH = 224
IMG_HEIGHT = 224

base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.25),
    Dense(2, activation='softmax')     # 정답은 'cat', 'dog' 2개이므로 출력층 노드는 2개
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 64)                131136    
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 20,992,746
Trainable params: 20,938,218
Non-trainable params: 54,528
_____________________________________

In [12]:
train_dir = './datafile/kagglecatsanddogs_5340/train'
test_dir = './datafile/kagglecatsanddogs_5340/validation'

train_data_gen = ImageDataGenerator(rescale=1./255, 
                                    rotation_range=10, width_shift_range=0.1, 
                                    height_shift_range=0.1, shear_range=0.1, zoom_range=0.1)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [17]:
train_data = train_data_gen.flow_from_directory(train_dir, batch_size=16, 
                                                color_mode='rgb', shuffle=True, class_mode='categorical', 
                                                target_size=(IMG_WIDTH, IMG_HEIGHT))
test_data = test_data_gen.flow_from_directory(test_dir, batch_size=16, 
                                              color_mode='rgb', shuffle=True, class_mode='categorical', 
                                              target_size=(IMG_WIDTH, IMG_HEIGHT))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [18]:
print(train_data.class_indices.items())
print(test_data.class_indices.items())

print(len(train_data.classes))
print(len(test_data.classes))

print(train_data.num_classes)
print(test_data.num_classes)

dict_items([('cats', 0), ('dogs', 1)])
dict_items([('cats', 0), ('dogs', 1)])
2000
1000
2
2


In [19]:
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(2e-5), 
              metrics=['accuracy'])

In [20]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

save_file_name = './cats_and_dogs_filtered_Xception_Colab.h5'

checkpoint = ModelCheckpoint(save_file_name, monitor='val_loss', 
                             verbose=2, save_best_only=True, mode='auto')

earlystopping = EarlyStopping(monitor='val_loss', patience=5)

hist = model.fit(train_data, epochs=30, validation_data=test_data, 
                 callbacks=[checkpoint, earlystopping])

Epoch 1/30
 77/125 [=================>............] - ETA: 8s - loss: 0.1984 - accuracy: 0.9497

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000016A1DDB5AE0>
Traceback (most recent call last):

  File "z:\롤체 덱 이길 확률 높이는 AI 만들기 project\TF2.x_YouTube\tfgpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "z:\롤체 덱 이길 확률 높이는 AI 만들기 project\TF2.x_YouTube\tfgpu\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "z:\롤체 덱 이길 확률 높이는 AI 만들기 project\TF2.x_YouTube\tfgpu\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "z:\롤체 덱 이길 확률 높이는 AI 만들기 project\TF2.x_YouTube\tfgpu\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "z:\롤체 덱 이길 확률 높이는 AI 만들기 project\TF2.x_YouTube\tfgpu\lib

In [ ]:
import matplotlib.pyplot as plt

